# Chapter 19. Dynamic Attributes ans Properties
`attributes`
- data attributes
- methods (`callables`)
- properties (as public data attributes)
    - `accessor methods`
    - `Uniform access principle`
        - All services offered by a module should be available through a uniform notation, which does not betray whether they are implemented through storage or through computation.

In [20]:
from urllib.request import urlopen
import warnings
import os
import json

URL = 'http://www.oreilly.com/pub/sc/osconfeed'
JSON = '/Users/tsubasa/VSCodeInsider/python3/20191230_fluent_python/osconfeed.json'

def load():
    if not os.path.exists(JSON):
        msg = 'downloading {} to {}'.format(URL, JSON)
        warnings.warn(msg)
        with urlopen(URL) as remote, open(JSON, 'wb') as local:
            local.write(remote.read())
    with open(JSON) as fp:
        return json.load(fp)

In [17]:
feed = load()

In [4]:
sorted(feed['Schedule'].keys())

['conferences', 'events', 'speakers', 'venues']

In [5]:
for key, value in sorted(feed['Schedule'].items()):
    print('{:3} {}'.format(len(value), key))

  1 conferences
494 events
357 speakers
 53 venues


In [6]:
feed['Schedule']['speakers'][-1]['name']

'Carina C. Zona'

In [7]:
feed['Schedule']['speakers'][-1]['serial']

141590

In [8]:
feed['Schedule']['events'][40]['name']

'There *Will* Be Bugs'

In [9]:
feed['Schedule']['events'][40]['speakers']

[3471, 5199]

In [10]:
from collections import abc


class FrozenJSON:
    def __init__(self, mapping):
        self.__data = dict(mapping)
    
    def __getattr__(self, name):
        if hasattr(self.__data, name):
            return getattr(self.__data, name)
        else:
            return FrozenJSON.build(self.__data[name])

    @classmethod
    def build(cls, obj):
        if isinstance(obj, abc.Mapping):
            return cls(obj)
        elif isinstance(obj, abc.MutableSequence):
            return [cls.build(item) for item in obj]
        else:
            return obj

## Flexible Object Creation with `__new__`

In [ ]:
# pseudo-code for object construction
def object_make(the_class, some_arg):
    new_object = tha_class.__new__(some_arg)
    if isinstance(new_object, the_class):
        the_class.__init__(new_object, some_arg)
    return new_object

In [11]:
from collections import abc


class FrozenJSON:
    def __new__(cls, arg):
        if isinstance(arg, abc.Mapping):
            return super().__new__(cls)
        elif isinstance(arg, abc.MutableSequence):
            return [cls(item) for item in arg]
        else:
            return arg
    
    def __init__(self, mapping):
        self.__data = {}
        for key, value in mapping.items():
            if iskeyword(key):
                key += '_'
            self.__data[key] = value
    
    def __getattr__(self, name):
        if hasattr(self.__data, name):
            return getattr(self.__data, name)
        else:
            return FrozenJSON(self.__data[name])

## Restructurinng the OSCON Feed with shelve

In [21]:
DB_NAME = '/Users/tsubasa/VSCodeInsider/python3/20191230_fluent_python/schedule1_db'
CONFERENCE = 'conference.115'

class Record:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)
    
def load_db(db):
    raw_data = load()
    warnings.warn('loading ' + DB_NAME)
    for collection, rec_list in raw_data['Schedule'].items():
        record_type = collection[:-1]
        for record in rec_list:
            key = '{}.{}'.format(record_type, record['serial'])
            record['serial'] = key
            db[key] = Record(**record)

In [22]:
import shelve
db = shelve.open(DB_NAME)
if CONFERENCE not in db:
    load_db(db)
speaker = db['speaker.3471']
type(speaker)

/Users/tsubasa/.pyenv/versions/3.7.2/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: loading /Users/tsubasa/VSCodeInsider/python3/20191230_fluent_python/schedule1_db
  # Remove the CWD from sys.path while we load stuff.


__main__.Record

In [23]:
speaker.name, speaker.twitter

('Anna Ravenscroft', 'annaraven')

In [24]:
db.close()

## Linked record Retrieval with Properties

In [ ]:
import warnings
import inspect

class Record:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)
    
    def __eq__(self, other):
        if isinstane(other, Record):
            return self.__dict__ == other.__dict__
        else:
            return NotImplemented

class MissingDatabaseError(RuntimeError):
    """Raised when a database is required bu wa not set."""

class DbRecord(Record):
    __db = None
    
    @staticmethod
    def set_db(db):
        DbRecord.__db = db
    
    @staticmethod
    def get_db():
        return DbRecord.__db
    
    @classmethod
    def fetch(cls, ident):
        db  =cls.get_db()
        try:
            return db[ident]
        except TypeError:
            if db is None:
                msg = "database not set; call '{}.set_db(my_db)'"
                raise MissingDatabaseError(msg.format(cls.__name__))
            else:
                raise
    
    def __repr__(self):
        if hasattr(self, 'serial'):
            cls_name = self.__class__.__name__
            return '<{} serial={!r}>'.format(cls_name, self.serial)
        else:
            return super().__repr__()

class Event(DbRecord):
    @property
    def venue(self):
        key = 'venue.{}'.format(self.venue_serial)
        return self.__class__.fetch(key)
    
    @property
    def speakers(self):
        if not hasattr(self, '_speaker_objs'):
            spkr_serials = self.__dict__['speakers']
            fetch = self.__class__.fetch
            self._speaker_objs = [fetch('speaker.{}'.format(key)) for key in spkr_serials]
        return self._speaker_objs
    
    def __repr__(self):
        if hasattr(self, 'name'):
            cls_name = self.__class__.__name__
            return '<{} {!r}'.format(cls_name, self.name)
        else:
            return super().__repr__()
    
    
    def load_db(db):
        raw_data = load()
        warnings.warn('loading ' + DB_NAME)
        for collection, rec_list in raw_data['Schedule'].items():
            record_type = collection[:-1]
            cls_name = record_type.capitalize()
            cls = globals().get(cls_name, DbRecord)
            if inspect.isclass(cls) and issubclass(cls, DbRecord):
                factory = cls
            else:
                factory = DbRecord
            for record in rec_list:
                key = '{}.{}'.format(record_type, record['serial'])
                record['serial'] = key
                db[key] = factory(**record)

# Using a Property for Attribute Validation

## LineItem Take #2: A Validating Property

In [ ]:
class LIneItem:
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
    
    def subtotal(self):
        return self.weight * self.price
    
    @property
    def weight(self):
        return self.__weight
    
    @weight.setter
    def weight(self, value):
        if value > 0:
            self.__weight = value
        else:
            raise ValueError('value must be > 0')

# A Proper Look at Properties

## Properties Override Instance Attributes
Search flow when `obj.attr`
1. `obj.__class__.attr`
1. `obj.attr`

In [25]:
class Class:
    data = 'the class data attr'
    @property
    def prop(self):
        return 'the prop value'

In [26]:
obj = Class()
vars(obj)

{}

In [27]:
obj.data

'the class data attr'

In [29]:
obj.data = 'bar'
vars(obj)

{'data': 'bar'}

In [30]:
obj.data

'bar'

In [31]:
Class.data

'the class data attr'

In [32]:
Class.prop

In [33]:
obj.prop

'the prop value'

In [34]:
obj.prop = 'foo'

AttributeError: can't set attribute

In [35]:
obj.__dict__['prop'] = 'foo'
vars(obj)

{'data': 'bar', 'prop': 'foo'}

In [36]:
obj.prop

'the prop value'

In [37]:
Class.prop = 'baz'
obj.prop

'foo'

# Coding a Property Factory

In [ ]:
def quantity(storage_name):
    def qty_getter(instance):
        return instance.__dict__[storage_name]
    
    def qty_setter(instance, value):
        if value > 0:
            instance.__dict__[storage_name] = value
        else:
            raise ValueError('value must be > 0')
    
    return property(qty_getter, qty_setter)


class LineItem:
    weight = quantity('weight')
    price = quantity('price')
    
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
    
    def subtotal(self):
        return self.weight * self.price

In [44]:
nutmeg = LineItem('Moluccan nutmeg', 8, 13.95)

# Essential Attributes and Functions for Attribute Handling

## Special Methods for Attribute Handling
- `__getattr__(self, name)`
    - Called only when an attempt to retrieve the named attribute failse, after the obj, Class, and its superclasses are searched.
    - i.e, only invoked after `__getattribute__` and only when `__getattribute__` raise `AttributeError`
- `__getattribute__(self, name)`
    - Always called when thee is an attempt to retrieve the named attribute, except when the attribute sought is a special attribute or method
- `__setattr__(self, name, value)`
    - Always called when there is an attempt to set the named attribute

<br>

It's better to use properties or descriptors because handling correctly these special methods is difficult.